In [1]:
import numpy as np 
import pandas as pd 
import gc
import os
import matplotlib.pyplot as plt
import json
import glob
from tqdm.notebook import tqdm

In [2]:
train = pd.read_json('train.json',lines=True)
test = pd.read_json('test.json',lines=True)
sub_sample = pd.read_csv('sample_submission.csv')

In [3]:
pretrain_dir = None
denoise=True
if denoise:
    train = train[train.signal_to_noise > 1].reset_index(drop = True)

In [4]:
test_pub = test[test["seq_length"] == 107]
test_pri = test[test["seq_length"] == 130]

In [5]:
As = []
for id in tqdm(train["id"]):
    a = np.load(f"bpps/{id}.npy")
    As.append(a)
As = np.array(As)
As_pub = []
for id in tqdm(test_pub["id"]):
    a = np.load(f"bpps/{id}.npy")
    As_pub.append(a)
As_pub = np.array(As_pub)
As_pri = []
for id in tqdm(test_pri["id"]):
    a = np.load(f"bpps/{id}.npy")
    As_pri.append(a)
As_pri = np.array(As_pri)

In [6]:
print(train.shape)
train.head()

(2096, 19)


,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
2,5,id_00ab2d761,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...,4.136,1,107,68,"[0.1942, 0.2041, 0.1626, 0.1213, 0.10590000000...","[0.2726, 0.2984, 0.21660000000000001, 0.1637, ...","[0.3393, 0.2728, 0.2005, 0.1703, 0.1495, 0.134...","[0.165, 0.20520000000000002, 0.179, 0.1333, 0....","[0.2864, 0.24710000000000001, 0.2222, 0.1903, ...","[0.7642, 1.6641, 1.0622, 0.5008, 0.4107, 0.133...","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[1.9554, 2.1298, 1.0403, 0.609, 0.5486, 0.386,...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0..."
3,6,id_00abef1d7,GGAAAACAAUUGCAUCGUUAGUACGACUCCACAGCGUAAGCUGUGG...,.........((((((((......((((((((((((....)))))))...,EEEEEEEEESSSSSSSSIIIIIISSSSSSSSSSSSHHHHSSSSSSS...,2.485,1,107,68,"[0.422, 0.5478000000000001, 0.4749000000000000...","[0.4801, 0.7943, 0.42160000000000003, 0.397300...","[0.9822000000000001, 1.272, 0.6940000000000001...","[0.5827, 0.7555000000000001, 0.5949, 0.4511, 0...","[0.9306000000000001, 1.0496, 0.5844, 0.7796000...","[0.895, 2.3377, 2.2305, 2.003, 1.9006, 1.0373,...","[0.46040000000000003, 3.6695, 0.78550000000000...","[2.7711, 7.365, 1.6924000000000001, 1.43840000...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1..."
4,7,id_00b436dec,GGAAAUCAUCGAGGACGGGUCCGUUCAGCACGCGAAAGCGUCGUGA...,.....(((((((((((..(((((((((..((((....))))..)))...,EEEEESSSSSSSSSSSIISSSSSSSSSIISSSSHHHHSSSSIISSS...,1.727,1,107,68,"[0.4843, 0.5233, 0.4554, 0.43520000000000003, ...","[0.8719, 1.0307, 0.6649, 0.34500000000000003, ...","[0.7045, 0.7775000000000001, 0.5662, 0.4561, 0...","[0.384, 0.723, 0.4766, 0.30260000000000004, 0....","[0.7429, 0.9137000000000001, 0.480400000000000...","[1.1576, 1.5137, 1.3382, 1.5622, 1.2121, 0.295...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.8641, 2.3767, 1.149, 1.0132, 0.9876, 0.0, 0...","[0.49060000000000004, 4.6339, 1.95860000000000...","[1.2852000000000001, 2.5460000000000003, 0.234..."


In [7]:
print(test.shape)
test.head()

(3634, 7)


,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [8]:
print(sub_sample.shape)
sub_sample.head()

(457953, 6)


,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0


In [9]:
targets = list(sub_sample.columns[1:])
print(targets)

y_train = []
seq_len = train["seq_length"].iloc[0]
seq_len_target = train["seq_scored"].iloc[0]
ignore = -10000
ignore_length = seq_len - seq_len_target
for target in targets:
    y = np.vstack(train[target])
    dummy = np.zeros([y.shape[0], ignore_length]) + ignore
    y = np.hstack([y, dummy])
    y_train.append(y)
y = np.stack(y_train, axis = 2)
y.shape

['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']


(2096, 107, 5)

In [10]:
def get_structure_adj(train):
    Ss = []
    for i in tqdm(range(len(train))):
        seq_length = train["seq_length"].iloc[i]
        structure = train["structure"].iloc[i]
        sequence = train["sequence"].iloc[i]

        cue = []
        a_structures = {
            ("A", "U") : np.zeros([seq_length, seq_length]),
            ("C", "G") : np.zeros([seq_length, seq_length]),
            ("U", "G") : np.zeros([seq_length, seq_length]),
            ("U", "A") : np.zeros([seq_length, seq_length]),
            ("G", "C") : np.zeros([seq_length, seq_length]),
            ("G", "U") : np.zeros([seq_length, seq_length]),
        }
        a_structure = np.zeros([seq_length, seq_length])
        
        for i in range(seq_length):
            if structure[i] == "(":
                cue.append(i)
            elif structure[i] == ")":
                start = cue.pop()
               
                a_structures[(sequence[start], sequence[i])][start, i] = 1
                a_structures[(sequence[i], sequence[start])][i, start] = 1
        
        a_strc = np.stack([a for a in a_structures.values()], axis = 2)
        a_strc = np.sum(a_strc, axis = 2, keepdims = True)
        Ss.append(a_strc)
    
    Ss = np.array(Ss)
    print(Ss.shape)
    return Ss

    
Ss = get_structure_adj(train)
Ss_pub = get_structure_adj(test_pub)
Ss_pri = get_structure_adj(test_pri)


(2096, 107, 107, 1)



(629, 107, 107, 1)



(3005, 130, 130, 1)


In [11]:
def get_distance_matrix(As):
    idx = np.arange(As.shape[1])
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1/Ds
    Ds = Ds[None, :,:]
    Ds = np.repeat(Ds, len(As), axis = 0)
    
    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis = 3)
    print(Ds.shape)
    return Ds

Ds = get_distance_matrix(As)
Ds_pub = get_distance_matrix(As_pub)
Ds_pri = get_distance_matrix(As_pri)

(2096, 107, 107, 3)
(629, 107, 107, 3)
(3005, 130, 130, 3)


In [12]:
As.shape

(2096, 107, 107)

In [13]:

As = np.concatenate([As[:,:,:,None], Ss, Ds], axis = 3).astype(np.float32)
As_pub = np.concatenate([As_pub[:,:,:,None], Ss_pub, Ds_pub], axis = 3).astype(np.float32)
As_pri = np.concatenate([As_pri[:,:,:,None], Ss_pri, Ds_pri], axis = 3).astype(np.float32)
del Ss, Ds, Ss_pub, Ds_pub, Ss_pri, Ds_pri
As.shape, As_pub.shape, As_pri.shape

((2096, 107, 107, 5), (629, 107, 107, 5), (3005, 130, 130, 5))

In [14]:
def return_ohe(n, i):
    tmp = [0] * n
    tmp[i] = 1
    return tmp

def get_input(train):
    mapping = {}
    vocab = ["A", "G", "C", "U"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_node = np.stack(train["sequence"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    

    mapping = {}
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_loop = np.stack(train["predicted_loop_type"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    mapping = {}
    vocab = [".", "(", ")"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_structure = np.stack(train["structure"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    X_node = np.concatenate([X_node, X_loop, X_structure], axis = 2)
    
    a = np.sum(X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]), axis = 2)
    
    vocab = sorted(set(a.flatten()))
    print(vocab)
    ohes = []
    for v in vocab:
        ohes.append(a == v)
    ohes = np.stack(ohes, axis = 2)
    X_node = np.concatenate([X_node, ohes], axis = 2).astype(np.float32)
    
    
    print(X_node.shape)
    return X_node


X_node = get_input(train)
X_node_pub = get_input(test_pub)
X_node_pri = get_input(test_pri)

[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(2096, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(629, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(3005, 130, 39)


# Model

In [15]:
import tensorflow as tf
from tensorflow.keras import layers as L
import tensorflow_addons as tfa
from tensorflow.keras import backend as K

def mcrmse(t, p, seq_len_target = seq_len_target):
    score = np.mean(np.sqrt(np.mean((p - y_va) ** 2, axis = 2))[:, :seq_len_target])
    return score

def mcrmse_loss(t, y, seq_len_target = seq_len_target):
    t = t[:, :seq_len_target]
    y = y[:, :seq_len_target]
    
    loss = tf.reduce_mean(tf.sqrt(tf.reduce_mean((t - y) ** 2, axis = 2)))
    return loss

def attention(x_inner, x_outer, n_factor, dropout):
    x_Q =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_inner)
    x_K =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_V =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_KT = L.Permute((2, 1))(x_K)
    res = L.Lambda(lambda c: K.batch_dot(c[0], c[1]) / np.sqrt(n_factor))([x_Q, x_KT])
#     res = tf.expand_dims(res, axis = 3)
#     res = L.Conv2D(16, 3, 1, padding = "same", activation = "relu")(res)
#     res = L.Conv2D(1, 3, 1, padding = "same", activation = "relu")(res)
#     res = tf.squeeze(res, axis = 3)
    att = L.Lambda(lambda c: K.softmax(c, axis=-1))(res)
    att = L.Lambda(lambda c: K.batch_dot(c[0], c[1]))([att, x_V])
    return att


def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor, dropout)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head, dropout) for i in range(n_head)]
        att = L.Concatenate()(heads)
        att = L.Dense(n_factor, 
                      kernel_initializer='glorot_uniform',
                      bias_initializer='glorot_uniform',
                     )(att)
    x = L.Add()([x, att])
    x = L.LayerNormalization()(x)
    if dropout > 0:
        x = L.Dropout(dropout)(x)
    return x


def res(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.LeakyReLU()(h)
    h = L.Dropout(rate)(h)
    return L.Add()([x, h])


def forward(x, unit, kernel = 3, rate = 0.1):
#     h = L.Dense(unit, None)(x)
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.Dropout(rate)(h)
#         h = tf.keras.activations.swish(h)
    h = L.LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h



def adj_attn(x, adj, unit, n = 2, rate = 0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = tf.matmul(adj, x_a)
        x_as.append(x_a)
    if n == 1:
        x_a = x_as[0]
    else:
        x_a = L.Concatenate()(x_as)
    x_a = forward(x_a, unit)
    return x_a


def get_base(config):
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")
    
    adj_learned = L.Dense(1, "relu")(adj)
    adj_all = L.Concatenate(axis = 3)([adj, adj_learned])
        
    xs = []
    xs.append(node)
    x1 = forward(node, 128, kernel = 3, rate = 0.0)
    x2 = forward(x1, 64, kernel = 6, rate = 0.0)
    x3 = forward(x2, 32, kernel = 15, rate = 0.0)
    x4 = forward(x3, 16, kernel = 30, rate = 0.0)
    x = L.Concatenate()([x1, x2, x3, x4])
    
    for unit in [64, 32]:
        x_as = []
        for i in range(adj_all.shape[3]):
            x_a = adj_attn(x, adj_all[:, :, :, i], unit, rate = 0.0)
            x_as.append(x_a)
        x_c = forward(x, unit, kernel = 30)
        
        x = L.Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)
        
    x = L.Concatenate()(xs)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    return model

def get_ae_model(base, config):
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")

    x = base([L.SpatialDropout1D(0.3)(node), adj])
    x = forward(x, 64, rate = 0.3)
    p = L.Dense(X_node.shape[2], "sigmoid")(x)
    
    loss = - tf.reduce_mean(20 * node * tf.math.log(p + 1e-4) + (1 - node) * tf.math.log(1 - p + 1e-4))
    model = tf.keras.Model(inputs = [node, adj], outputs = [loss])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = lambda t, y : y)
    return model

def get_model(base, config):
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")
    
    x = base([node, adj])
    x = forward(x, 128, rate = 0.4)
    x = L.Dense(5, None)(x)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = mcrmse_loss)
    return model

def get_optimizer():
    adam = tf.optimizers.Adam()

    return adam

In [16]:

ae_epochs = 50
ae_epochs_each = 10
ae_batch_size = 32

epochs_list = [50, 30, 8, 8, 15, 15]
batch_size_list = [8, 16, 32, 64, 128, 256]


In [17]:
config = {}

if ae_epochs > 0:
    base = get_base(config)
    ae_model = get_ae_model(base, config)
    
    for i in range(ae_epochs//ae_epochs_each):
        print(f"------ {i} ------")
        print("--- train ---")
        ae_model.fit([X_node, As], [X_node[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        print("--- public ---")
        ae_model.fit([X_node_pub, As_pub], [X_node_pub[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        print("--- private ---")
        ae_model.fit([X_node_pri, As_pri], [X_node_pri[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        gc.collect()
    print("****** save ae model ******")
    base.save_weights("./base_ae")

------ 0 ------
--- train ---
Epoch 1/10
66/66 [==============================] - 16s 244ms/step - loss: 0.9493
Epoch 2/10
66/66 [==============================] - 14s 205ms/step - loss: 0.3419
Epoch 3/10
66/66 [==============================] - 14s 205ms/step - loss: 0.1734
Epoch 4/10
66/66 [==============================] - 14s 205ms/step - loss: 0.1129
Epoch 5/10
66/66 [==============================] - 14s 209ms/step - loss: 0.0851
Epoch 6/10
66/66 [==============================] - 13s 204ms/step - loss: 0.0675
Epoch 7/10
66/66 [==============================] - 13s 204ms/step - loss: 0.0566
Epoch 8/10
66/66 [==============================] - 14s 205ms/step - loss: 0.0496
Epoch 9/10
66/66 [==============================] - 13s 204ms/step - loss: 0.0414
Epoch 10/10
66/66 [==============================] - 14s 205ms/step - loss: 0.0416
--- public ---
Epoch 1/10
20/20 [==============================] - 7s 362ms/step - loss: 0.0341
Epoch 2/10
20/20 [==============================] - 4

In [18]:
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle = True, random_state = 42)

scores = []
preds = np.zeros([len(X_node), X_node.shape[1], 5])
for i, (tr_idx, va_idx) in enumerate(kfold.split(X_node, As)):
    
    print(f"------ fold {i} start -----")
    X_node_tr = X_node[tr_idx]
    X_node_va = X_node[va_idx]
    As_tr = As[tr_idx]
    As_va = As[va_idx]
    y_tr = y[tr_idx]
    y_va = y[va_idx]
    
    base = get_base(config)
    if ae_epochs > 0:
        
        base.load_weights("./base_ae")
    model = get_model(base, config)
    if pretrain_dir is not None:
        d = f"./model{i}"
        print(f"--- load from {d} ---")
        model.load_weights(d)
    for epochs, batch_size in zip(epochs_list, batch_size_list):
        print(f"epochs : {epochs}, batch_size : {batch_size}")
        model.fit([X_node_tr, As_tr], [y_tr],
                  validation_data=([X_node_va, As_va], [y_va]),
                  epochs = epochs,
                  batch_size = batch_size, validation_freq = 3)
        
    model.save_weights(f"./model{i}")
    p = model.predict([X_node_va, As_va])
    scores.append(mcrmse(y_va, p))
    print(f"fold {i}: mcrmse {scores[-1]}")
    preds[va_idx] = p
    
    
        
pd.to_pickle(preds, "oof.pkl")

------ fold 0 start -----
------ fold 0 start -----
------ fold 0 start -----
****** load ae model ******
epochs : 50, batch_size : 8
Epoch 1/50
210/210 [==============================] - 22s 104ms/step - loss: 0.4800
Epoch 2/50
210/210 [==============================] - 19s 92ms/step - loss: 0.2551
Epoch 3/50
210/210 [==============================] - 27s 127ms/step - loss: 0.2300 - val_loss: 0.2116
Epoch 4/50
210/210 [==============================] - 19s 92ms/step - loss: 0.2180
Epoch 5/50
210/210 [==============================] - 19s 92ms/step - loss: 0.2082
Epoch 6/50
210/210 [==============================] - 21s 100ms/step - loss: 0.2024 - val_loss: 0.1939
Epoch 7/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1976
Epoch 8/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1932
Epoch 9/50
210/210 [==============================] - 21s 100ms/step - loss: 0.1903 - val_loss: 0.1840
Epoch 10/50
210/210 [==============================] - 19s

Epoch 3/30
105/105 [==============================] - 15s 143ms/step - loss: 0.1413 - val_loss: 0.1590
Epoch 4/30
105/105 [==============================] - 14s 130ms/step - loss: 0.1409
Epoch 5/30
105/105 [==============================] - 14s 130ms/step - loss: 0.1407
Epoch 6/30
105/105 [==============================] - 15s 143ms/step - loss: 0.1400 - val_loss: 0.1586
Epoch 7/30
105/105 [==============================] - 14s 131ms/step - loss: 0.1404
Epoch 8/30
105/105 [==============================] - 14s 130ms/step - loss: 0.1398
Epoch 9/30
105/105 [==============================] - 15s 142ms/step - loss: 0.1397 - val_loss: 0.1590
Epoch 10/30
105/105 [==============================] - 14s 130ms/step - loss: 0.1392
Epoch 11/30
105/105 [==============================] - 14s 131ms/step - loss: 0.1393
Epoch 12/30
105/105 [==============================] - 15s 142ms/step - loss: 0.1392 - val_loss: 0.1594
Epoch 13/30
105/105 [==============================] - 14s 131ms/step - loss: 0.1

14/14 [==============================] - 8s 561ms/step - loss: 0.1239
Epoch 11/15
14/14 [==============================] - 8s 561ms/step - loss: 0.1240
Epoch 12/15
14/14 [==============================] - 9s 617ms/step - loss: 0.1240 - val_loss: 0.1643
Epoch 13/15
14/14 [==============================] - 8s 560ms/step - loss: 0.1241
Epoch 14/15
14/14 [==============================] - 8s 560ms/step - loss: 0.1238
Epoch 15/15
14/14 [==============================] - 9s 615ms/step - loss: 0.1241 - val_loss: 0.1642
epochs : 15, batch_size : 256
Epoch 1/15
7/7 [==============================] - 7s 991ms/step - loss: 0.1241
Epoch 2/15
7/7 [==============================] - 7s 993ms/step - loss: 0.1235
Epoch 3/15
7/7 [==============================] - 8s 1s/step - loss: 0.1233 - val_loss: 0.1640
Epoch 4/15
7/7 [==============================] - 7s 994ms/step - loss: 0.1232
Epoch 5/15
7/7 [==============================] - 7s 989ms/step - loss: 0.1230
Epoch 6/15
7/7 [=========================

210/210 [==============================] - 19s 92ms/step - loss: 0.1562
Epoch 33/50
210/210 [==============================] - 21s 101ms/step - loss: 0.1562 - val_loss: 0.1643
Epoch 34/50
210/210 [==============================] - 19s 93ms/step - loss: 0.1551
Epoch 35/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1542
Epoch 36/50
210/210 [==============================] - 21s 101ms/step - loss: 0.1535 - val_loss: 0.1656
Epoch 37/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1527
Epoch 38/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1526
Epoch 39/50
210/210 [==============================] - 21s 101ms/step - loss: 0.1510 - val_loss: 0.1649
Epoch 40/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1515
Epoch 41/50
210/210 [==============================] - 19s 92ms/step - loss: 0.1504
Epoch 42/50
210/210 [==============================] - 21s 101ms/step - loss: 0.1501 - val_loss: 0.1660
Epoch 43

7/7 [==============================] - 8s 1s/step - loss: 0.1220 - val_loss: 0.1604
Epoch 13/15
7/7 [==============================] - 7s 990ms/step - loss: 0.1220
Epoch 14/15
7/7 [==============================] - 7s 994ms/step - loss: 0.1219
Epoch 15/15
7/7 [==============================] - 8s 1s/step - loss: 0.1220 - val_loss: 0.1603
fold 4: mcrmse 0.16027487640676225


In [19]:
print(scores)

[0.16232385027155818, 0.15810400089962118, 0.1638799233738299, 0.16551536349430412, 0.16027487640676225]


# Prediction

In [20]:
p_pub = 0
p_pri = 0
for i in range(5):
    model.load_weights(f"./model{i}")
    p_pub += model.predict([X_node_pub, As_pub]) / 5
    p_pri += model.predict([X_node_pri, As_pri]) / 5
    

for i, target in enumerate(targets):
    test_pub[target] = [list(p_pub[k, :, i]) for k in range(p_pub.shape[0])]
    test_pri[target] = [list(p_pri[k, :, i]) for k in range(p_pri.shape[0])]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [21]:
preds_ls = []
for df, preds in [(test_pub, p_pub), (test_pri, p_pri)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=targets)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)


In [25]:
preds_df[(preds_df['reactivity'] < -0.5) | (preds_df['deg_50C'] < -0.5) | (preds_df['deg_Mg_50C'] < -0.5) 
           | (preds_df['deg_pH10'] < -0.5) | (preds_df['deg_Mg_pH10'] < -0.5)  ]

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id_seqpos


In [23]:
preds_df['deg_Mg_pH10'][(preds_df['deg_Mg_pH10'] < -0.5)]=-0.5
preds_df['deg_pH10'][(preds_df['deg_pH10'] < -0.5)]=-0.5
preds_df['deg_Mg_50C'][(preds_df['deg_Mg_50C'] < -0.5)]=-0.5
preds_df['deg_50C'][(preds_df['deg_50C'] < -0.5)]=-0.5

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [24]:
preds_df.to_csv("submission_v3.csv", index = False)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id_seqpos
0,0.847710,0.603009,1.743468,0.501308,0.740290,id_00073f8be_0
1,2.389182,3.087172,4.334582,3.137658,2.724101,id_00073f8be_1
2,1.712926,0.598558,0.748595,0.756142,0.683571,id_00073f8be_2
3,1.257642,0.893702,1.051911,1.435852,1.599114,id_00073f8be_3
4,0.758058,0.502518,0.488114,0.806599,0.811167,id_00073f8be_4
